In [1]:
from termios import TIOCM_DSR
import warnings
import random
import json
import sys
import cv2
import tqdm
import os

from ai2thor.controller import Controller
import ai2thor
from allenact_plugins.ithor_plugin.ithor_util import (
    horizontal_to_vertical_fov,
)
from data_collection_utils import (
    pos_to_id,
    teleport_to,
    take_step,
    find_shortes_terminal_path
)
from boolset.tasks_and_samplers import (
    AgentPose,
    HouseAugmenter,
    ProcTHORDataset,
    Vector3,
)

import prior
from allenact.embodiedai.sensors.vision_sensors import DepthSensor

# configurations
THOR_COMMIT_ID = "627521a56508f212749a779d358ab17df10e0d8e"
CAMERA_WIDTH = 224
CAMERA_HEIGHT = 224
HORIZONTAL_FIELD_OF_VIEW = 100
STEP_SIZE = 0.2
ROTATION_DEGREES = 45.0
VISIBILITY_DISTANCE = 1.5

dataset = prior.load_dataset("procthor-10k")
train_scenes = dataset["train"]
val_scenes = dataset["val"]
SCENES = ProcTHORDataset(
    [i for i in train_scenes] + [i for i in val_scenes]
)

# define dataset collection parameters
TRAIN = False if sys.argv[1] == "test" else True
NUM_ANCHORS = 1000 if TRAIN else 100
NUM_STEPS = 4
ROT_ANGLE = 30
ACTIONS = ["MoveAhead", "MoveBack", "RotateLeft", "RotateRight"]
IMG_ROOT = '../data/interactron/train' if TRAIN else '../data/interactron/test'
ANN_PATH = '../data/interactron/annotations/interactron_v1_train.json' if TRAIN \
    else '../data/interactron/annotations/interactron_v1_test.json'
CTRL = Controller(
    commit_id=THOR_COMMIT_ID,
    server_class=ai2thor.fifo_server.FifoServer,
    include_private_scenes=False,
    fastActionEmit=True,
    snapToGrid=False,
    autoSimulation=False,
    autoSyncTransforms=True,
    width=CAMERA_WIDTH,
    height=CAMERA_HEIGHT,
    fieldOfView=horizontal_to_vertical_fov(
        horizontal_fov_in_degrees=HORIZONTAL_FIELD_OF_VIEW,
        width=CAMERA_WIDTH,
        height=CAMERA_HEIGHT,
    ),
    makeAgentsVisible=True,
    visibilityScheme="Distance",
    agentMode="arm",
    rotateStepDegrees=ROTATION_DEGREES,
    visibilityDistance=VISIBILITY_DISTANCE,
    gridSize=STEP_SIZE,
    useMassThreshold=True,
    massThreshold=10,
    platform="CloudRendering",
    renderSemanticSegmentation=True
)

/home/hzhu2/anaconda3/envs/allenact-foundation/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading test: 100%|██████████| 1000/1000 [00:00<00:00, 9281.04it/s]


In [2]:
currect_house = SCENES.initialize_house(
    controller=CTRL,
    index=0 # [0, len(SCENES)]
)


In [3]:
import numpy as np
current_house_ind = 0
for i in range(10):
    standing = (
        {}
        if CTRL.initialization_parameters["agentMode"]
        == "locobot"
        else {"standing": np.random.choice([False, True])}
    )
    starting_pose: AgentPose = {
        "position": np.random.choice(
            SCENES.reachable_positions_map[  # type: ignore
                current_house_ind
            ]
        ),
        "rotation": Vector3(
            x=0, y=np.random.random() * 360, z=0
        ),
        "horizon": np.random.randint(-1, 2) * 30,
        "standing": standing["standing"]
        if "standing" in standing
        else None,
    }

    md = CTRL.step(
        action="TeleportFull", **starting_pose
    ).metadata
    if not md["lastActionSuccess"]:
        if i == 10 - 1:
            warnings.warn(
                f"Teleport failed in {current_house_ind} {10} times!"
            )
        continue
    break
else:
    raise Exception("Could not find a valid starting pose.")

In [9]:
for i in md["objects"]:
    print(i["objectId"], i["breakable"], i["mass"],
          i["axisAlignedBoundingBox"]["size"])

{'cornerPoints': [[6.102851867675781, 2.1074485778808594, 5.524420738220215], [6.102851867675781, 2.1074485778808594, 4.486842155456543], [6.102851867675781, 0.0005974769592285156, 5.524420738220215], [6.102851867675781, 0.0005974769592285156, 4.486842155456543], [5.867778778076172, 2.1074485778808594, 5.524420738220215], [5.867778778076172, 2.1074485778808594, 4.486842155456543], [5.867778778076172, 0.0005974769592285156, 5.524420738220215], [5.867778778076172, 0.0005974769592285156, 4.486842155456543]], 'center': {'x': 5.985315322875977, 'y': 1.054023027420044, 'z': 5.005631446838379}, 'size': {'x': 0.23507308959960938, 'y': 2.106851100921631, 'z': 1.0375785827636719}}


TypeError: float() argument must be a string or a number, not 'dict'

In [12]:
md = CTRL.step("randomizeObjectMass").metadata